In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

import xgboost as xgb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster
import shap


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../data/Top100_prediction_con.csv')
print(df.shape)
df.sample(1)

(1739323, 12)


,Unnamed: 0,id,store,date,sales,item,store_code,yearweek,sell_price,weekday,event,income
161335,7149626,SUPERMARKET_3_546_BOS_1,South_End,2011-09-20,5,SUPERMARKET_3_546,BOS_1,201138,7.176,Tuesday,0.0,35.88


In [3]:
lista_productos100 = list(df['item'].unique())

In [4]:
resultados = []

# Copia del DataFrame original
df_original = df.copy()

# Seleccionar los primeros 5 productos únicos
productos = list(df_original['item'].unique())

for i in lista_productos100:
        dataset = df_original[['item','date','income','weekday','event']]
        df_product = dataset[dataset['item']==i]
        df_product_sin_duplicates = df_product[['date','weekday', 'event']].drop_duplicates()
        ts_predict = df_product.groupby(['date'])['income'].sum().reset_index()
        ts_predict = ts_predict.merge(df_product_sin_duplicates, on=['date'], how='left')
        ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
        ts_predict['date'] = pd.to_datetime(ts_predict['date'])
        ts_predict.sort_values('date', ascending=True, inplace=True)
        ts_predict.set_index('date', inplace=True)
        steps = 30
        ts_predict = ts_predict.asfreq('D')
        y, exog = ts_predict['income'], ts_predict.drop(columns=['income'])
        y_train, y_test = y[:-30], y[-30:]
        exog_train, exog_test = exog[:-30], exog[-30:]
        
        forecaster = ForecasterAutoreg(
                    regressor = RandomForestRegressor(random_state=123,
                                                        max_depth=10,
                                                        min_samples_leaf=2, 
                                                        min_samples_split=5,
                                                        n_estimators=50),
                    lags      =  7
                )
        
        start_date = '2016-04-25' 
        end_date ='2016-05-30'
        # Crear un rango de fechas
        date_range = pd.date_range(start=start_date, end=end_date)
        # Crear el dataframe
        df1 = pd.DataFrame(date_range, columns=['date'])
        df1['date'] = pd.to_datetime(df1['date'])
        df1['weekday'] = df1['date'].dt.day_name()
        df1['event'] = 0
        df1 = pd.get_dummies(data=df1, columns=['weekday'], dtype=int)
        df1.sort_values('date', inplace=True, ascending=True)
        df1.set_index('date', inplace=True)
        df1 = df1.asfreq('D')
        exog_test = pd.concat([exog_test, df1], ignore_index=False)

        forecaster.fit(y=y_train, exog=exog_train)
        predictions = forecaster.predict(steps=60, exog=exog_test).to_list()
        resultados.append({'i':i, 'prediction':predictions,'test':y_test.values})
    



/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider 

In [5]:
tiendas_inventario = pd.DataFrame(resultados)
tiendas_inventario['test_suma'] = tiendas_inventario['test'].apply(sum)
suma = tiendas_inventario['test_suma'].sum()
tiendas_inventario['prediction'] = tiendas_inventario['prediction'].apply(lambda x: np.round(x, decimals=0))
tiendas_inventario['tiendas_inventario_test'] = tiendas_inventario['prediction'].apply(lambda x: sum(x[:30]))
tiendas_inventario['tiendas_inventario_prediction'] = tiendas_inventario['prediction'].apply(lambda x: sum(x[30:]))
tiendas_inventario[['i','test_suma','tiendas_inventario_test','tiendas_inventario_prediction']]

,i,test_suma,tiendas_inventario_test,tiendas_inventario_prediction
0,ACCESORIES_1_108,4592.0644,5245.0,4673.0
1,HOME_&_GARDEN_1_027,4656.6000,5546.0,5781.0
2,HOME_&_GARDEN_1_053,18937.0500,15807.0,15319.0
3,HOME_&_GARDEN_1_140,9075.5625,7454.0,7064.0
4,HOME_&_GARDEN_1_177,4153.3625,7384.0,7008.0
...,...,...,...,...
95,SUPERMARKET_3_499,10380.2880,8727.0,9226.0
96,ACCESORIES_1_158,49910.0322,45690.0,44603.0
97,SUPERMARKET_3_282,18462.7200,17146.0,16787.0
98,ACCESORIES_1_354,48962.5668,53037.0,53145.0


In [6]:
total_inventario_productos_real = tiendas_inventario['test_suma'].sum()
total_inventario_productos_predicho = tiendas_inventario['tiendas_inventario_test'].sum()
print(total_inventario_productos_real)
print(total_inventario_productos_predicho)
print(total_inventario_productos_real - total_inventario_productos_predicho)

1086598.199
1057787.0
28811.199000000022


In [7]:
ventas_globales = pd.DataFrame(resultados) ##prueba de prediccion

In [9]:
file_name = "ventas_global.xlsx"
ventas_globales.to_excel(file_name)

: 

In [11]:
pd.DataFrame(resultados)

,i,prediction,test
0,ACCESORIES_1_108,"[219.6863901605724, 208.7791345989689, 190.600...","[172.63400000000001, 224.4242, 69.0536, 172.63..."
1,HOME_&_GARDEN_1_027,"[201.48006109565415, 188.1308466352049, 163.73...","[216.41249999999997, 171.63749999999996, 194.0..."
2,HOME_&_GARDEN_1_053,"[651.090699319551, 723.3965300824177, 582.0362...","[804.6375, 879.4875, 748.5, 392.9625, 411.6749..."
3,HOME_&_GARDEN_1_140,"[443.70170791909896, 307.78379404761904, 256.4...","[299.4, 411.67499999999995, 130.9875, 168.4125..."
4,HOME_&_GARDEN_1_177,"[313.0052414355889, 318.34362242044784, 240.93...","[331.07500000000005, 357.21250000000003, 217.8..."
...,...,...,...
95,SUPERMARKET_3_499,"[225.7398714417077, 223.5834158974359, 265.332...","[178.176, 116.736, 181.248, 218.112, 282.624, ..."
96,ACCESORIES_1_158,"[1541.8807341033796, 1960.1857135928942, 1759....","[2200.5648, 1711.5504, 1956.0576, 1589.2968, 1..."
97,SUPERMARKET_3_282,"[643.3726918510473, 763.3869543802092, 573.719...","[844.8, 909.312, 559.104, 466.944, 497.664, 47..."
98,ACCESORIES_1_354,"[1698.0163092320956, 1919.3586900211883, 1730....","[2139.438, 1375.353, 1344.7896, 1375.353, 1436..."


: 